In [1]:
from sqlalchemy import create_engine, text
import pandas as pd 
from datetime import datetime, timedelta

In [2]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

In [23]:
with engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS companies'))

2024-04-16 17:36:08,008 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 17:36:08,008 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS companies
2024-04-16 17:36:08,008 INFO sqlalchemy.engine.Engine [generated in 0.00387s] ()
2024-04-16 17:36:08,039 INFO sqlalchemy.engine.Engine ROLLBACK


## Companies 

In [24]:
create_company_table = text('''CREATE TABLE IF NOT EXISTS companies (
    company_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER UNIQUE NOT NULL,
    name VARCHAR(255) NOT NULL,
    industry VARCHAR(255),
    address TEXT,
    contact_email VARCHAR(255),
    website URL,
    description TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);''')

In [25]:
# Execute SQL statements
with engine.connect() as connection:
    connection.execute(create_company_table)

2024-04-16 17:36:13,365 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 17:36:13,366 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS companies (
    company_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER UNIQUE NOT NULL,
    name VARCHAR(255) NOT NULL,
    industry VARCHAR(255),
    address TEXT,
    contact_email VARCHAR(255),
    website URL,
    description TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);
2024-04-16 17:36:13,367 INFO sqlalchemy.engine.Engine [cached since 108.9s ago] ()
2024-04-16 17:36:13,398 INFO sqlalchemy.engine.Engine ROLLBACK


In [26]:
pd.read_sql('SELECT * FROM companies',engine)

2024-04-16 17:36:16,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 17:36:16,131 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM companies")
2024-04-16 17:36:16,131 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 17:36:16,135 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM companies")
2024-04-16 17:36:16,135 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 17:36:16,136 INFO sqlalchemy.engine.Engine SELECT * FROM companies
2024-04-16 17:36:16,137 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 17:36:16,139 INFO sqlalchemy.engine.Engine ROLLBACK


,company_id,user_id,name,industry,address,contact_email,website,description


In [32]:
def add_company(user_id, name, industry, address, contact_email, website, description):
    # SQL query to insert a new company
    insert_query = text("""
        INSERT INTO companies (user_id, name, industry, address, contact_email, website, description)
        VALUES (:user_id, :name, :industry, :address, :contact_email, :website, :description);
    """)
    
    # Execute the query within a transaction
    with engine.begin() as connection:
        connection.execute(insert_query, {
            "user_id": user_id,
            "name": name,
            "industry": industry,
            "address": address,
            "contact_email": contact_email,
            "website": website,
            "description": description
        })
        print(f"Company {name} added successfully.")


def delete_company(company_id):
    delete_query = text("DELETE FROM companies WHERE company_id = :company_id")

    with engine.begin() as connection:
        result = connection.execute(delete_query, {'company_id': company_id})
        if result.rowcount > 0:
            print(f"Company with ID {company_id} deleted successfully.")
        else:
            print(f"No company found with ID {company_id}, nothing deleted.")


def modify_company(company_id, **kwargs):
    updates = [f"{key} = :{key}" for key in kwargs]
    update_query = text(f"UPDATE companies SET {', '.join(updates)} WHERE company_id = :company_id")

    kwargs['company_id'] = company_id  # Adding company_id to the parameters for the WHERE clause

    with engine.begin() as connection:
        result = connection.execute(update_query, kwargs)
        if result.rowcount > 0:
            print(f"Company with ID {company_id} updated successfully.")
        else:
            print(f"No company found with ID {company_id}, nothing updated.")

In [33]:
pd.read_sql('SELECT * FROM companies',engine)

2024-04-16 17:41:19,201 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 17:41:19,202 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM companies")
2024-04-16 17:41:19,202 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 17:41:19,203 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM companies")
2024-04-16 17:41:19,203 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 17:41:19,205 INFO sqlalchemy.engine.Engine SELECT * FROM companies
2024-04-16 17:41:19,206 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 17:41:19,206 INFO sqlalchemy.engine.Engine ROLLBACK


,company_id,user_id,name,industry,address,contact_email,website,description
0,1,1,Tech Innovations Inc.,Technology,"123 Tech Drive, Innovation City",contact@techinnovations.com,http://www.techinnovations.com,A leading company in tech innovations aiming t...


In [ ]:
# Example usage
add_company(
    user_id=1, 
    name='Tech Innovations Inc.', 
    industry='Technology', 
    address='123 Tech Drive, Innovation City', 
    contact_email='contact@techinnovations.com', 
    website='http://www.techinnovations.com', 
    description='A leading company in tech innovations aiming to...'
)